In [1]:
import numpy as np
import pandas as pd

In [113]:
# pandas中的时间解析是很灵活的
datestrs = ['2019-07-06 12:00:00', '1/09/2019', '20190101', 'Jul 31, 2019', np.datetime64('2018-01-01'), datetime.datetime.now()]
pd.to_datetime(datestrs)

DatetimeIndex([       '2019-07-06 12:00:00',        '2019-01-09 00:00:00',
                      '2019-01-01 00:00:00',        '2019-07-31 00:00:00',
                      '2018-01-01 00:00:00', '2019-09-01 10:29:35.118640'],
              dtype='datetime64[ns]', freq=None)

---
在pandas的时间序列中需要明确几个概念：

| 概念       | 标量类  | 数组类      | pandas数据类型                   | 创建方法                    |
| ------------ | ---------- | -------------- | ------------------------------------ | ------------------------------- |
| Date times   | Timestamp  | DatetimeIndex  | datetime64[ns] or datetime64[ns, tz] | to_datetime or date_range       |
| Time deltas  | Timedelta  | TimedeltaIndex | timedelta64[ns]                      | to_timedelta or timedelta_range |
| Time spans   | Period     | PeriodIndex    | period[freq]                         | Period or period_range          |
| Date offsets | DateOffset | None           | None                                 | DateOffset                      |


- Date times：具有时区支持的特定日期和时间。与标准库中的datetime.datetime类似。

- Time deltas：绝对持续时间。与标准库中的datetime.timedelta类似。

- Time spans：由时间点及其相关频率定义的时间跨度。

- Date offsets：尊重日历算术的相对持续时间。 与dateutil包中的dateutil.relativedelta.relativedelta类似。

---
# 1、Date times

## 1.1 创建

### 1.1.1 to_datetime
`pd.to_datetime(arg, errors='raise', dayfirst=False, yearfirst=False, utc=None, box=True, format=None, exact=True, unit=None, infer_datetime_format=False, origin='unix', cache=True)`

- arg：integer, float, string, datetime, list, tuple, 1-d array, Series or DataFrame/dict-like
- errors：{'ignore', 'raise', 'coerce'}, 默认为 'raise'
    - 如果为ignore，遇到无法解析的字符串会返回原字符串
    - 如果为raise，遇到无法解析的字符串会抛出异常
    - 如果为coerce，遇到无法解析的字符串会转为NaT
- dayfirst：指定解析顺序（如果 arg 参数为字符串或类似于列表的对象）。如果为True，10/11/12 会被解析为 2012/11/10
- yearfirst：指定解析顺序（如果 arg 参数为字符串或类似于列表的对象）。如果为True，10/11/12 会被解析为 2010/11/12
    - 如果 dayfirst 和 yearfirst 都为True的话，yearfirst优先级高（默认）
- format：指定解析格式。
    - pd.to_datetime('12-2010-10 00:00', format='%d-%Y-%m %H:%M') 会被解析为 2010-10-12 00:00:00
    
返回类型依赖于输入：
- 输入标量，返回Timestap
- 输入数组，返回DatetimeIndex
- 输入一个Series/DataFrame，返回Series

In [84]:
# 输入标量，返回Timestap
pd.to_datetime('2019')

Timestamp('2019-01-01 00:00:00')

In [86]:
# 输入数组，返回DatetimeIndex
pd.to_datetime(['20190101', '20190201', '20190301'])

DatetimeIndex(['2019-01-01', '2019-02-01', '2019-03-01'], dtype='datetime64[ns]', freq=None)

In [87]:
# 输入一个Series，返回Series
s = pd.Series(['20190101', '20190201', '20190301'])
pd.to_datetime(s)

0   2019-01-01
1   2019-02-01
2   2019-03-01
dtype: datetime64[ns]

也可以通过DataFrame来创建时间序列，但是需要通过列名称来指定时间单位：

 `year`, `month`, `day`是必选列名
 
 `hour` , `minute`, `second`, `millisecond`, `microsecond`, `nanosecond`是可选列名

In [106]:
# 输入一个DataFrame，返回一个Series
df = pd.DataFrame({'year': [2018, 2019],'month': [3, 4], 'day': [6, 8],'hour': [3, 1], 'minute': [10, 20]})
pd.to_datetime(df)

pd.to_datetime(df[['year', 'month', 'day']])

0   2018-03-06 03:10:00
1   2019-04-08 01:20:00
dtype: datetime64[ns]

0   2018-03-06
1   2019-04-08
dtype: datetime64[ns]

---
### 1.1.2 date_range
`pd.date_range(start=None, end=None, periods=None, freq=None, tz=None, normalize=False, name=None, closed=None, **kwargs)`

生成固定频率的`DatetimeIndex`

- start：string或datetime-like，默认值是None，表示日期的起点
- end：string或datetime-like，默认值是None，表示日期的终点
- periods：integer或None，默认值是None，表示你要从这个函数产生多少个日期索引值；如果是None的话，那么start和end必须不能为None
- freq：string或DateOffset，默认值是’D’，表示以自然日为单位，这个参数用来指定计时单位，比如’5H’表示每隔5个小时计算一次。在[这里](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases)可以看到所有可选值。
    - Y 表示年
    - M 表示月
    - D 表示日
    - W 表示周
    - H 表示时
    - T 表示分
    - S 表示秒
    - B 表示工作日
- tz：string或None，表示时区，例如：’Asia/Hong_Kong’
- normalize：bool，默认值为False，如果为True的话，那么在产生时间索引值之前会先把start和end都转化为当日的午夜0点
- name：str，默认值为None，给返回的时间索引指定一个名字
- closed：string或者None，默认值为None，表示start和end这个区间端点是否包含在区间内，可以有三个值，’left’表示左闭右开区间，’right’表示左开右闭区间，None表示两边都是闭区间


`start`, `end`, `periods`, `freq`这四个参数至少需要指定三个，其中`freq`默认为D

In [96]:
pd.date_range('2019-01-01', periods=3, freq='T') # freq 默认为D，可以写成3D，表示间隔为3天

DatetimeIndex(['2019-01-01 00:00:00', '2019-01-01 00:01:00',
               '2019-01-01 00:02:00'],
              dtype='datetime64[ns]', freq='T')

In [98]:
pd.date_range('20190101', periods=4, freq='10T')

DatetimeIndex(['2019-01-01 00:00:00', '2019-01-01 00:10:00',
               '2019-01-01 00:20:00', '2019-01-01 00:30:00'],
              dtype='datetime64[ns]', freq='10T')

---
### 1.1.3 bdate_range
`pd.bdate_range(start=None, end=None, periods=None, freq='B', tz=None, normalize=True, name=None, weekmask=None, holidays=None, closed=None, **kwargs)`

按照工作日计算

In [111]:
pd.bdate_range(start='2018-01-01', end='2019-01-01')

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05', '2018-01-08', '2018-01-09', '2018-01-10',
               '2018-01-11', '2018-01-12',
               ...
               '2018-12-19', '2018-12-20', '2018-12-21', '2018-12-24',
               '2018-12-25', '2018-12-26', '2018-12-27', '2018-12-28',
               '2018-12-31', '2019-01-01'],
              dtype='datetime64[ns]', length=262, freq='B')

In [112]:
pd.date_range(start='2018-01-01', end='2019-01-01')

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08',
               '2018-01-09', '2018-01-10',
               ...
               '2018-12-23', '2018-12-24', '2018-12-25', '2018-12-26',
               '2018-12-27', '2018-12-28', '2018-12-29', '2018-12-30',
               '2018-12-31', '2019-01-01'],
              dtype='datetime64[ns]', length=366, freq='D')

## 1.2 索引

In [122]:
d = pd.date_range('20180101', '20190601')


In [60]:
# 其它时间序列相关的函数
tt = pd.Timestamp('20190101')
tt.day_name()

pd.Timedelta('1 d')


# 缺失值用pd.NaT表示

'Tuesday'

Timedelta('1 days 00:00:00')